# Cell Tracking and Lineage Construction in Live-Cell Imaging Data

### Part 2 (of 2)
---

## Employing the Model to Track Cells and Benchmarking the Results
---

Implementation of:

[Accurate cell tracking and lineage construction in live-cell imaging experiments with deep learning](https://www.biorxiv.org/content/10.1101/803205v2)

Deployed at:

[DeepCell.org](http://www.deepcell.org/)

### Global Imports

In [1]:
import os
import datetime
import errno
import argparse
import fnmatch
import logging

import numpy as np

from skimage.external.tifffile import imsave
from skimage.external.tifffile import TiffFile

import deepcell
from deepcell.utils.misc_utils import sorted_nicely
from deepcell.utils.tracking_utils import load_trks

## Track Multiple Movies with Multiple Parameters

### Load Ground Truth Benchmark Data

In [2]:
# Download four different sets of ground truth data (saves to ~/.keras/datasets)
# We will re-track this GT data to verify our results

filename_3T3 = '3T3_NIH_benchmarks.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.nih_3t3_bench.load_tracked_data(filename_3T3)
print('3T3 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_HEK = 'HEK293_benchmarks.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.hek293_bench.load_tracked_data(filename_HEK)
print('HEK293 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_HeLa = 'HeLa_S3_benchmarks.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.hela_s3_bench.load_tracked_data(filename_HeLa)
print('HeLa -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_RAW = 'RAW2647_benchmarks.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.raw2647_bench.load_tracked_data(filename_RAW)
print('RAW264.7 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

322985984/322979840 [==============================] - 14s 0us/step
3T3 -
X.shape: (19, 30, 154, 182, 1)
y.shape: (19, 30, 154, 182, 1)
134987776/134983680 [==============================] - 6s 0us/step
HEK293 -
X.shape: (20, 30, 135, 160, 1)
y.shape: (20, 30, 135, 160, 1)
637083648/637081600 [==============================] - 24s 0us/step
HeLa -
X.shape: (14, 40, 216, 256, 1)
y.shape: (14, 40, 216, 256, 1)
226959360/226959360 [==============================] - 11s 0us/step
RAW264.7 -
X.shape: (10, 30, 202, 240, 1)
y.shape: (10, 30, 202, 240, 1)


### Setup File Path Constants

In [3]:
# Change DATA_DIR if you are not using `deepcell.datasets`
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models'))

# Define destination folders for tracked output
TRACK_DIRS = [
    os.path.abspath(os.path.join(DATA_DIR, 'track_data/3T3')),
    os.path.abspath(os.path.join(DATA_DIR, 'track_data/HEK293')),
    os.path.abspath(os.path.join(DATA_DIR, 'track_data/HeLa')),
    os.path.abspath(os.path.join(DATA_DIR, 'track_data/RAW264.7'))
]

# create directories if they do not exist
for d in TRACK_DIRS + [MODEL_DIR]:
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [4]:
# Define data to track (benchmark test files)
# Each trks file contains multiple movies (or batches)

GT_BASE_DIR = '/data/npz_data/tracking_benchmark_data/test/'

GT_trks_files = [
    os.path.join(GT_BASE_DIR, '3T3_NIH_test_BData.trks'),
    os.path.join(GT_BASE_DIR, 'HEK293_generic_test_BData.trks'),
    os.path.join(GT_BASE_DIR, 'HeLa_S3_test_BData.trks'),
    os.path.join(GT_BASE_DIR, 'RAW264_generic_test_BData.trks')
]

assert all(os.path.exists(x) for x in GT_trks_files)

### Work Through Each Dataset with Multiple Parameters

#### Import a Tracking Model to Use

In [5]:
from deepcell import model_zoo

# Tracking model settings (These settings should mirror those from Part 1)
features = {'appearance', 'distance', 'neighborhood', 'regionprop'}

min_track_length = 9
neighborhood_scale_size = 30
batch_size = 128  
crop_dim = 32

# Re-instantiate the tracking model
tracking_model = model_zoo.siamese_model(
    input_shape=(crop_dim, crop_dim, 1),
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

# Load model weights
siamese_weights_file = 'tracking_model_seed1_tl9.h5'
siamese_weights_file = os.path.join(MODEL_DIR, siamese_weights_file)

tracking_model.load_weights(siamese_weights_file)

#### Define Tracking Parameters

In [6]:
# Define Critical Parameters 
# Key variables are lists to facilitate parameter sweeps if desired

# If model prediction for daughter is higher than this parameter,
# then daughter assignment made
division = [0.9]

# Parameter for cost matrix - if other possible assignments are higher than this,
# then a birth occurred
birth = [0.99]

# Parameter for cost matrix - if other possible assignments are higher than this,
# then a death occurred
death = [0.99]

# If two labels are beyond this distance they will not be compared
# (smaller distances -> faster tracking)
max_distance = 50

# Define Parameters that should not be changed
# (they are model dependent or training data dependent)
track_length = 9
neighborhood_scale_size = 30

#### Run the Model Iteratively for the Parameters of Interest

In [7]:
# The tracking model is used in concert with other processes to track cells
# Import the neccesary tracking functionality
import deepcell.tracking


# The tracker requires normalized images as input
def image_norm(img):
    return (img - np.mean(img)) / np.std(img)


# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(GT_trks_files):
    trks = load_trks(dataset)
    # Go through each batch (movie) in each dataset
    for batch_num, (lineage, raw, tracked) in enumerate(zip(trks['lineages'], trks['X'], trks['y'])):        
        # Normalize raw images
        for frame in range(raw.shape[0]):
            raw[frame, :, :, 0] = image_norm(raw[frame, :, :, 0]) 

        # Track each movie with each division parameter
        print('Tracking: Dataset {}, Batch {}'.format(set_num, batch_num))
        for div_param in division:
            # For each birth parameter
            for birth_param in birth:
                # For each death parameter
                for death_param in death:
                    pass
                    # Track with the selected parameters
                    trial = deepcell.tracking.cell_tracker(
                        raw, tracked, tracking_model,
                        max_distance=max_distance,
                        track_length=track_length,
                        division=div_param,
                        birth=birth_param,
                        death=death_param,
                        neighborhood_scale_size=neighborhood_scale_size,
                        features=features)

                    trial.track_cells()
                    
                    # Run FP post processing and save the result in the correct location
                    file_name = 'batch_{}_div{}_b{}_d{}'.format(
                        str(batch_num).zfill(3),
                        str(div_param).zfill(3),
                        str(birth_param).zfill(3),
                        str(death_param).zfill(3))
                    file_path = os.path.join(TRACK_DIRS[set_num], file_name)
                    fp_fixed_trk = trial.postprocess(file_path)


Tracking: Dataset 0, Batch 0
Tracking: Dataset 0, Batch 1
Tracking: Dataset 0, Batch 2
Tracking: Dataset 0, Batch 3
Tracking: Dataset 0, Batch 4
Tracking: Dataset 0, Batch 5
Tracking: Dataset 0, Batch 6
Tracking: Dataset 0, Batch 7
Tracking: Dataset 0, Batch 8
Tracking: Dataset 0, Batch 9
Tracking: Dataset 0, Batch 10
Tracking: Dataset 0, Batch 11
Tracking: Dataset 0, Batch 12
Tracking: Dataset 0, Batch 13
Tracking: Dataset 0, Batch 14
Tracking: Dataset 0, Batch 15
Tracking: Dataset 0, Batch 16
Tracking: Dataset 0, Batch 17
Tracking: Dataset 0, Batch 18
Tracking: Dataset 0, Batch 19
Tracking: Dataset 0, Batch 20
Tracking: Dataset 0, Batch 21
Tracking: Dataset 0, Batch 22
Tracking: Dataset 0, Batch 23
Tracking: Dataset 1, Batch 0
Tracking: Dataset 1, Batch 1
Tracking: Dataset 1, Batch 2
Tracking: Dataset 1, Batch 3
Tracking: Dataset 1, Batch 4
Tracking: Dataset 1, Batch 5
Tracking: Dataset 1, Batch 6
Tracking: Dataset 1, Batch 7
Tracking: Dataset 1, Batch 8
Tracking: Dataset 1, Batch 9


## Translate the Results Into ISBI Format for Comparison

#### Save Cell Lineages in an ISBI-Formatted Output txt

The ISBI Cell Tracking Challenge requires a text file (man_track.txt) that represents a batch's cell lineage as an acyclic graph. The format of this file is as follows: Every line corresponds to a single track that is encoded by four numbers separated by a space -  
L B E P  
where L is a unique label of the track (label of markers, 16-bit positive value),  
B is a zero-based index of the frame in which the track begins,  
E is a zero-based index of the frame in which the track ends,  
P is the label of the parent track (0 is used when no parent is defined)

N.B. DeepCell's unique approach allows for cells to be tracked even if it momentarily leaves the frame. This is not possible in convential tracking algorithms, so ISBI considers a cell's track to have ended once it leaves the frame. We adjust the output here to keep with ISBI's formatting (ie. each track only contains contiguous frames).

#### ISBI Translator Functions

In [8]:
from deepcell_tracking.isbi_utils import contig_tracks
from deepcell_tracking.isbi_utils import trk_to_isbi

#### Define Save Location for Benchmark Files (ie: 001, 001_GT, 001_RES)

In [9]:
# Define where benchmark data will be saved
BENCH_DIR = os.path.abspath(os.path.join(DATA_DIR, 'tracking_benchmarks'))

# Define where tracks will be saved
BENCH_DIRS = [
    os.path.join(BENCH_DIR, '3T3'),
    os.path.join(BENCH_DIR, 'HEK293'),
    os.path.join(BENCH_DIR, 'HeLa'),
    os.path.join(BENCH_DIR, 'RAW264.7')
]

# create directories if they do not exist
for d in BENCH_DIRS:
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

#### Generate RAW and GT Benchmark Files

In [10]:
# Create benchmark files (ie: 001, 001_GT, etc) from the Ground Truth .trk files

from skimage.external.tifffile import imsave

for set_num, dataset in enumerate(GT_trks_files):
    # Load trks file
    trks = load_trks(dataset)
    lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]

    # Define Save Location
    PARAMS = 'div{}_b{}_d{}'.format(
        str(div_param).zfill(3),
        str(birth_param).zfill(3),
        str(death_param).zfill(3),
    )

    # Build subdirectories to hold benchmark info           
    BENCHMARK_DIR = os.path.abspath(os.path.join(BENCH_DIRS[set_num], PARAMS))
    
    # First loop through tracks and ensure that all tracks have continuous frames.
    for batch, batch_info in enumerate(lineages):

        batch_tracked = tracked[batch]
        labels = list(batch_info.keys())
        max_label = max(labels)

        for label in labels:
            batch_info, batch_tracked = contig_tracks(label, batch_info, batch_tracked)

            if max(batch_info.keys()) > max_label:
                # New track was added!
                new_max_label = max(batch_info.keys())
                labels.append(new_max_label)
                max_label = new_max_label

        tracked[batch] = batch_tracked  # resave inside original array

    # Record lineage data in txt as it is generated
    for batch, batch_info in enumerate(lineages):
        # Build subdirectories to hold benchmark info
        # Build subdirectories to hold benchmark info
        B_SUB_DIR_RAW = os.path.join(BENCHMARK_DIR, '{:03}'.format(batch + 1))
        B_SUB_DIR_GT = os.path.join(BENCHMARK_DIR, '{:03}_GT'.format(batch + 1))
        B_SUB_DIR_SEG = os.path.join(B_SUB_DIR_GT, 'SEG')
        B_SUB_DIR_TRA = os.path.join(B_SUB_DIR_GT, 'TRA')

        # Create directories if they do not exist
        for d in (B_SUB_DIR_RAW, B_SUB_DIR_GT, B_SUB_DIR_SEG, B_SUB_DIR_TRA):
            try:
                os.makedirs(d)
            except OSError as exc:  # Guard against race condition
                if exc.errno != errno.EEXIST:
                    raise

        # Resave the .trk image data with new track values.
        channel = 0 # These images should only have one channel
        for i in range(raw.shape[1]):
            name_raw = os.path.join(B_SUB_DIR_RAW,'t{:03}_.tif'.format(i))
            name_tracked_SEG = os.path.join(B_SUB_DIR_SEG,'man_seg{:03}.tif'.format(i))
            name_tracked_TRA = os.path.join(B_SUB_DIR_TRA,'man_track{:03}.tif'.format(i))
            imsave(name_raw, raw[batch, i, :, :, channel])
            imsave(name_tracked_SEG, tracked[batch, i, :, :, channel].astype('uint16'))
            imsave(name_tracked_TRA, tracked[batch, i, :, :, channel].astype('uint16'))

        # Prepare output txt
        filename = os.path.join(B_SUB_DIR_TRA, "man_track.txt")
        trk_to_isbi(batch_info, filename)


#### Translate Result trk files and Generate Benchmark Files

In [11]:
## Create new benchmark files (ie: 001_RES) from mutliple .trk files

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(TRACK_DIRS):
    # Define Save Location
    PARAMS = 'div{}_b{}_d{}'.format(
        str(div_param).zfill(3),
        str(birth_param).zfill(3),
        str(death_param).zfill(3))

    # Build subdirectories to hold benchmark info           
    BENCHMARK_DIR = os.path.abspath(os.path.join(BENCH_DIRS[set_num], PARAMS))

    # Go through each batch (movie) in each dataset
    movie_list = sorted_nicely(os.listdir(dataset))
    
    for batch_num, batch in enumerate(movie_list):
        # Load the trk file
        trks = load_trks(os.path.join(dataset, batch))
        lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]

        B_SUB_DIR = os.path.join(BENCHMARK_DIR, '{:03}_RES'.format(batch_num+1))
        print('Files will be saved at: ', B_SUB_DIR)

        # create directories if they do not exist
        try:
            os.makedirs(B_SUB_DIR)
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

        # Record lineage data in txt as it is generated
        batch_info = lineages[0]

        # Prepare output txt
        batch_tracked = tracked
        labels = list(batch_info.keys())
        max_label = max(labels)

        for label in labels:
            batch_info, tracked = contig_tracks(label, batch_info, tracked)

            if max(batch_info.keys()) > max_label:
                # New track was added!
                new_max_label = max(batch_info.keys())
                labels.append(new_max_label)
                max_label = new_max_label

        # Save Image Files
        channel = 0 # These images should only have one channel
        for i in range(raw.shape[0]):
            name_tracked = os.path.join(B_SUB_DIR,'mask{:03}.tif'.format(i))
            imsave(name_tracked, tracked[i, :, :, channel].astype('uint16'))

        filename = os.path.join(B_SUB_DIR, "res_track.txt")
        trk_to_isbi(batch_info, filename)

Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/001_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/002_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/003_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/004_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/005_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/006_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/007_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/008_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/009_RES
Files will be saved at:  /root/.keras/datasets/tracking_benchmarks/3T3/div0.9_b0.99_d0.99/010_RES
Files will be saved 

## Run Graph Benchmarks

#### Graph Comparison Code

In [12]:
from deepcell.metrics import match_nodes

#### Iteratively Compare Each Movie (Tracked with Each Parameter) To the Ground Truth Data

In [13]:
from deepcell_tracking.isbi_utils import classify_divisions
from deepcell_tracking.isbi_utils import txt_to_graph

In [14]:
import glob

import networkx as nx

from deepcell_tracking.utils import clean_up_annotations

# Prep a list to hold each dataset's results
dataset_cm = []

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for ben_trk_folder in BENCH_DIRS:
    print(ben_trk_folder)
    # Prep a list to hold results for each set of parameters
    params_cm = []
   
    # Go through each parameter combination
    for div_param in division:
        # For each birth parameter
        for birth_param in birth:
            # For each death parameter
            for death_param in death:  

                PARAMS = 'div{}_b{}_d{}'.format(div_param, birth_param, death_param)
                BENCHMARK_DIR = os.path.join(ben_trk_folder, PARAMS)

                # Compile a list of each movie
                sub_dirs = sorted_nicely(os.listdir(BENCHMARK_DIR))
                movie_list = fnmatch.filter(sub_dirs, '???')
                # Prep a list to hold each movie's results
                cm_list = []
                # Loop through each set of movies 
                for name in movie_list:
                    # Extract track.txt for each movie
                    pattern_gt = os.path.join(BENCHMARK_DIR, name + '_GT/TRA/')
                    pattern_res = os.path.join(BENCHMARK_DIR, name + '_RES/')

                    # Load gt and clean up to generate unique labels
                    gt = np.stack([TiffFile(f).asarray()
                                   for f in np.sort(glob.glob(pattern_gt + '*.tif'))])

                    res = np.stack([TiffFile(f).asarray()
                                    for f in np.sort(glob.glob(pattern_res + '*.tif'))])

                    unique = clean_up_annotations(np.copy(gt))

                    # Calculate iou matrix for each dataset
                    iou_gt = match_nodes(gt, unique)
                    iou_res = match_nodes(res, unique)

                    thresh = 0.5
                    x, y, z = np.where(iou_gt > thresh)
                    node_key_gt = {'{}_{}'.format(y[i], x[i]): '{}_{}'.format(z[i], x[i])
                                   for i in range(x.shape[0])}

                    x, y, z = np.where(iou_res > thresh)
                    node_key_res = {'{}_{}'.format(y[i], x[i]): '{}_{}'.format(z[i], x[i])
                                    for i in range(x.shape[0])}

                    G_res = txt_to_graph(pattern_res + 'res_track.txt')
                    G_gt = txt_to_graph(pattern_gt + 'man_track.txt')

                    G_res = nx.relabel.relabel_nodes(G_res, node_key_res)
                    G_gt = nx.relabel.relabel_nodes(G_gt, node_key_gt)

                    stats = classify_divisions(G_gt, G_res)
                    if any(stats[k] for k in ('Incorrect division', 'False negative division')):
                        print(name)
                    cm_list.append(stats)

                params_cm.append(cm_list)

    dataset_cm.append(params_cm)

/root/.keras/datasets/tracking_benchmarks/3T3
missed node 297_12 division completely
005
missed node 493_25 division completely
007
missed node 429_5 division completely
014
missed node 435_2 division completely
021
missed node 524_13 division completely
missed node 601_20 division completely
024
/root/.keras/datasets/tracking_benchmarks/HEK293
846_9 out degree = 2, daughters mismatch, gt and res degree equal.
002
missed node 843_13 division completely
005
missed node 732_13 division completely
006
missed node 1139_25 division completely
016
missed node 579_2 division completely
017
831_13 out degree = 1, daughters mismatch.
018
missed node 955_16 division completely
019
807_24 out degree = 1, daughters mismatch.
023
missed node 523_7 division completely
024
807_15 out degree = 3, daughters mismatch.
025
missed node 845_2 division completely
026
/root/.keras/datasets/tracking_benchmarks/HeLa
missed node 1304_36 division completely
005
/root/.keras/datasets/tracking_benchmarks/RAW264.7


In [15]:
# Tally up all the statistics
cm_totals = []
for cell_type in dataset_cm:
    cm_params = []
    for param_comb in cell_type:

        dataset_stats = {
            'Correct division': 0,
            'Incorrect division': 0,
            'False positive division': 0,
            'False negative division': 0
        }

        for cm in param_comb:
            for k in dataset_stats:
                dataset_stats[k] += cm[k]

        cm_params.append(dataset_stats)
    
    cm_totals.append(cm_params)

In [16]:
counter = 0
# Go through each parameter combination
for div_param in division:
    # For each birth parameter
    for birth_param in birth:
        # For each death parameter
        for death_param in death:

            PARAMS = 'div{}_b{}_d{}'.format(div_param, birth_param, death_param)
            print(PARAMS)
            for i, name in enumerate(['3T3', 'HEK293', 'HeLa', 'RAW']):
                print(name)
                print(cm_totals[i][counter])

            counter = counter + 1

div0.9_b0.99_d0.99
3T3
{'Correct division': 18, 'False negative division': 6, 'Incorrect division': 0, 'False positive division': 0}
HEK293
{'Correct division': 32, 'False negative division': 7, 'Incorrect division': 4, 'False positive division': 6}
HeLa
{'Correct division': 8, 'False negative division': 1, 'Incorrect division': 0, 'False positive division': 0}
RAW
{'Correct division': 21, 'False negative division': 1, 'Incorrect division': 0, 'False positive division': 4}


## Run ISBI Benchmarking Scripts 

#### ISBI Cell Tracking Challenge Software Required 

ISBI's Cell Tracking Challenge has a specific [evaluation methodolgy](http://celltrackingchallenge.net/evaluation-methodology/). The evaluation package can be downloaded [here](http://public.celltrackingchallenge.net/software/EvaluationSoftware.zip). Extract the TRAMeasure executable from the zip file and place it in the same directory as this notebook.

In [17]:
import subprocess
import statistics

# Confirm the ISBI CTC executable file is available
assert os.path.isfile('./TRAMeasure'),'CTC EXE unavailable. See subheading for download instructions.'

# Define path to folder containing the directories of interest (ie: 001, 001_GT, 001_RES)
bens_folder_names = ['3T3', 'HEK293', 'HeLa', 'RAW264.7']

for index, path in enumerate(BENCH_DIRS):
    
    # Check into parameter folder if neccesary
    PARAMS = 'div{}_b{}_d{}'.format(
        str(div_param).zfill(3),
        str(birth_param).zfill(3),
        str(death_param).zfill(3))

    path = os.path.abspath(os.path.join(path, PARAMS))

    # Calculate the number of batches
    dirs = os.listdir(path)
    num_batches = int(len(dirs)/3)

    TRA_Vals = []
    for batch in range(num_batches):
        batch = '{:03}'.format(batch+1)

        # Run ISBI Tracking Benchmark
        p = subprocess.run(['./TRAMeasure', path, batch], stdout=subprocess.PIPE)
        # Save the output
        outstring = p.stdout

        try:
            TRA_Val = float(outstring.decode('utf-8').split()[-1])
            TRA_Vals.append(TRA_Val)
        except:
            print('Benchmarking failure - Batch ', batch)
            print(outstring.decode('utf-8'))
    
    print('Dataset: ', bens_folder_names[index])
    print('Mean: ', statistics.mean(TRA_Vals))
    print('Std. Dev.: ', statistics.pstdev(TRA_Vals))

Dataset:  3T3
Mean:  0.9998178333333333
Std. Dev.:  0.0003087691514528082
Dataset:  HEK293
Mean:  0.9995048076923077
Std. Dev.:  0.0006110526238971484
Dataset:  HeLa
Mean:  0.9942584444444444
Std. Dev.:  0.021668905564380152
Dataset:  RAW264.7
Mean:  0.9996796923076923
Std. Dev.:  0.000320740341809237
